# (RAG) system built from scratch using Sentence-Transformers, FAISS, Cross-Encoder reranking, and an open-source LLM

## Install Dependencies & Imports & Setup

In [ ]:
!pip install pypdf
!pip install sentence-transformers
!pip install transformers
!pip install faiss-cpu
!pip install torch


In [ ]:
import os
import json
import time
import faiss
import numpy as np
import pandas as pd

from pathlib import Path
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from transformers import pipeline

## Load & Read Any File Type

In [ ]:

def load_documents(file_path: str):
    """
    Loads the content of .txt, .csv, .jsonl, or .pdf files and returns a list of text documents.
    Perfect for building a RAG corpus.
    """
    file_path = Path(file_path)
    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    ext = file_path.suffix.lower()

    # TXT
    if ext == ".txt":
        return [file_path.read_text(encoding="utf-8")]

    # CSV (first column = content)
    elif ext == ".csv":
        df = pd.read_csv(file_path)
        return df.iloc[:, 0].astype(str).tolist()

    # JSONL (each line = {"text": "..."} )
    elif ext == ".jsonl":
        docs = []
        with open(file_path, "r", encoding="utf-8") as f:
            for line in f:
                docs.append(json.loads(line).get("text", ""))
        return docs

    # PDF
    elif ext == ".pdf":
        reader = PdfReader(str(file_path))
        pages = [page.extract_text() for page in reader.pages]
        return [p for p in pages if p]   # remove empty pages

    else:
        raise ValueError(f"Unsupported file format: {ext}")


In [ ]:
# Load our Corpus

file_path = "/content/Big_Data.pdf"  # source pdf that will be use as a RAG knowledge base
corpus_texts = load_documents(file_path)

print(f"Loaded {len(corpus_texts)} documents from {file_path}")

## Embedding Model
We use `all-MiniLM-L6-v2`, a fast and lightweight embedding model ideal for retrieval

In [ ]:
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Convert all documents into dense vectors and normalize them for FAISS retrieval (Embeddings).

print("\n Embedding corpus...")

corpus_embeddings = embed_model.encode(
    corpus_texts,
    batch_size=32,
    convert_to_numpy=True,
    show_progress_bar=True
)

# FAISS requires normalized vectors for cosine similarity
faiss.normalize_L2(corpus_embeddings)

dim = corpus_embeddings.shape[1]
print(f"Vector dimension: {dim}")


## Build FAISS Index


In [ ]:
print("\n Building FAISS index...")

index = faiss.IndexFlatIP(dim)
index.add(corpus_embeddings)

print(f" FAISS index built with {index.ntotal} vectors.")

## Ask a Query → Retrieve Top-K


In [ ]:
query = "What are the charactristcs of big data? "
print("\n Your query:", query)

# Embed query
q_vec = embed_model.encode(query, convert_to_numpy=True).reshape(1, -1)
faiss.normalize_L2(q_vec)

# FAISS retrieve
scores, idx = index.search(q_vec, k=5)

retrieved_docs = []
for rank, doc_i in enumerate(idx[0]):
    retrieved_docs.append({
        "rank": rank + 1,
        "text": corpus_texts[doc_i]
    })

print("\nTop Retrieved Docs:")
display(pd.DataFrame(retrieved_docs))

##  Rerank with Cross Encoder (Better Precision)


In [ ]:
print("\n Reranking using CrossEncoder...")
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

pairs = [[query, d["text"]] for d in retrieved_docs]
rerank_scores = reranker.predict(pairs)

# Attach rerank score
for i, score in enumerate(rerank_scores):
    retrieved_docs[i]["rerank_score"] = float(score)

# Sort by cross-encoder
reranked = sorted(retrieved_docs, key=lambda x: x["rerank_score"], reverse=True)

print("\n Top Reranked Docs:")
display(pd.DataFrame(reranked))


## Build Final RAG Prompt
Assemble the top retrieved documents and generate a final instruction prompt for the LLM.


In [ ]:
top_docs = reranked[:3]  # use top-3 for LLM

context = "\n\n".join(
    [f"[Doc {i+1}] {d['text']}" for i, d in enumerate(top_docs)]
)

prompt = f"""
Use ONLY the following evidence:

{context}

Question: {query}

Answer clearly, scientifically, and with accurate facts.
"""

print("\n Final Prompt Sent to LLM:\n")
print(prompt[:600], "...")

## Generate Final Answer

In [ ]:
# Use Phi-3.5-mini-instruct (fast, small, reliable) to produce the final RAG answer

print("\n Generating answer...")

generator = pipeline(
    "text-generation",
    model="microsoft/Phi-3.5-mini-instruct",
    max_new_tokens=200,
    device=device # Explicitly set device to use GPU if available, otherwise CPU
)

answer = generator(prompt)[0]["generated_text"]

print("\n Final Answer:\n")
print(answer)